## Lidar com dados ausentes

O conjunto de dados pode ter valores ausentes por vários motivos, como observações não registradas ou corrupção de dados. A manipulação de dados ausentes é muito importante, pois muitos algoritmos analíticos não suportam dados com valores ausentes.

Os dados ausentes são representados como `NaN`, que significa "Not a Number" em pandas DataFrame. Demonstraremos técnicas comuns para lidar com valores ausentes em um DataFrame que incluem:
- Encontre informações gerais sobre valores ausentes no DataFrame
- Simplesmente solte linhas com valor ausente
- Preencha o valor ausente
     - com constante
     - com moda (para recursos categóricos)
     - com média/mediana (para características contínuas)
     - com valores estimados gerados a partir de outras colunas

Primeiro, criaremos um DataFrame com valores ausentes.

In [1]:
import numpy as np
import pandas as pd
df1 = pd.DataFrame({'LastName':['Mend', 'Brun', 'Lu', 'Guy'],
                    'FirstName':['Kim', 'Rob', 'Lin', 'Ron'],
                    'Gender':['F', 'M', 'M', np.NaN],
                    'Age':[20, 21, np.NaN, 22]})
df1

,LastName,FirstName,Gender,Age
0,Mend,Kim,F,20.0
1,Brun,Rob,M,21.0
2,Lu,Lin,M,NaN
3,Guy,Ron,NaN,22.0


Informações gerais sobre valores ausentes
O Pandas DataFrame tem uma função info() que imprimirá informações gerais do DataFrame, incluindo número de linhas, número de colunas, tipo de dados de cada coluna e número de valores não nulos em todas as colunas. A célula abaixo mostra que df1 tem 3 linhas, 4 colunas, LastName e FirstName têm 3 valores não nulos ou nenhum valor ausente, Genero e Idade têm 2 valores não nulos ou cada um tem um valor ausente.

In [ ]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   LastName   4 non-null      object 
 1   FirstName  4 non-null      object 
 2   Gender     3 non-null      object 
 3   Age        3 non-null      float64
dtypes: float64(1), object(3)
memory usage: 256.0+ bytes


In [7]:
df1.isna().sum() # Quantidade de valores NaN por coluna

LastName     0
FirstName    0
Gender       1
Age          1
dtype: int64

In [5]:
cols_with_nan = [column for column in df1.columns if df1[column].isna().any()]

print(f'Columns in df with NaN values: {cols_with_nan}')

Columns in df with NaN values: ['Gender', 'Age']


In [9]:
 # Visualizando quantos valores NaN há em cada coluna nas quais já sabemos que
 # há valores NaN
df1[cols_with_nan].isna().sum()

Gender    1
Age       1
dtype: int64

#### Simplesmente descarte as linhas com valores ausentes

A maneira mais simples de lidar com o valor ausente é simplesmente descartar as linhas que possuem valores ausentes em colunas específicas com a função DataFrame `dropna()`. A função por padrão retornará um novo DataFrame com o valor ausente descartado. Se você quiser modificar o DataFrame no local, você terá que definir `inplace=True`. Tente `help(df1.dropna)` para ver detalhes desta função.

In [11]:
df1.dropna?

In [10]:
#remover as linhas cuja idade não é especificada
df1_1 = df1.dropna(subset=['Age'])
df1_1

,LastName,FirstName,Gender,Age
0,Mend,Kim,F,20.0
1,Brun,Rob,M,21.0
3,Guy,Ron,NaN,22.0


#### Preencha os valores ausentes

Também podemos preencher valores ausentes com a função `fillna()`. Vamos definir o argumento da função `inplace=True` para preencher os valores ausentes no local.

Vamos demonstrar 3 maneiras de preencher os valores ausentes:

- Preencher com constante
- Preencher com média/moda
- Preencha com valores calculados com base em outra coluna

##### Preencher com constante
Preencha a idade ausente com 20.

In [12]:
df1.fillna?

In [13]:
# preencher as idades faltantes com 20
df1_2 = df1.copy()
df1_2.Age.fillna(20, inplace=True)
df1_2

,LastName,FirstName,Gender,Age
0,Mend,Kim,F,20.0
1,Brun,Rob,M,21.0
2,Lu,Lin,M,20.0
3,Guy,Ron,NaN,22.0


In [14]:
# preencher com a média da idade
df1_3 = df1.copy()
df1_3.Age.fillna(df1_3.Age.mean(), inplace=True)
df1_3

,LastName,FirstName,Gender,Age
0,Mend,Kim,F,20.0
1,Brun,Rob,M,21.0
2,Lu,Lin,M,21.0
3,Guy,Ron,NaN,22.0


In [15]:
# preencher o genero com a moda
"""
OBS: O método mode() retorna um iterável com inúmeras modas, em casos de séries
multimodais, ou com única moda. Para abranger casos gerais, selecionamos apenas
o elemento no primeiro índice do iterável (moda com maior frequência)
"""
df1_4 = df1.copy()
df1_4.Gender.fillna(df1_4.Gender.mode()[0], inplace=True)
df1_4

,LastName,FirstName,Gender,Age
0,Mend,Kim,F,20.0
1,Brun,Rob,M,21.0
2,Lu,Lin,M,NaN
3,Guy,Ron,M,22.0


In [16]:
df1

,LastName,FirstName,Gender,Age
0,Mend,Kim,F,20.0
1,Brun,Rob,M,21.0
2,Lu,Lin,M,NaN
3,Guy,Ron,NaN,22.0


In [19]:
df1.groupby('Gender').Age.mean()

Gender
F    20.0
M    21.0
Name: Age, dtype: float64

In [21]:
df1.groupby('Gender').Age.mean()['F']

20.0

In [22]:
df1.groupby('Gender').Age.mean()['M']

21.0

In [23]:
df1.groupby('Gender').Age.transform('mean')

0    20.0
1    21.0
2    21.0
3     NaN
Name: Age, dtype: float64

In [24]:
# preencher com base em outra coluna
df1_5 = df1.copy()
df1_5.Age.fillna(df1_5.groupby('Gender').Age.transform('mean'), inplace=True)
df1_5

,LastName,FirstName,Gender,Age
0,Mend,Kim,F,20.0
1,Brun,Rob,M,21.0
2,Lu,Lin,M,21.0
3,Guy,Ron,NaN,22.0


## Manipular valores de string

Existem inúmeras maneiras de transformar valores de string. Apresentaremos algumas manipulações comuns. Usaremos principalmente o atributo `str` da série pandas para invocar funções de string.

- Tira espaços em branco à esquerda/à direita
- Unificar maiúsculas e minúsculas (isso é especialmente crítico se você precisar comparar ou agrupar pelos valores na coluna)
- Substitua certos caracteres na string (ou seja, lidando com valores de moeda)
- converter string de data e hora em tipo de dados de data e hora.

Primeiro, vamos criar um DataFrame para demonstração acima das manipulações de strings. Vamos:
- retire os espaços em branco à esquerda/à direita na coluna Gênero e converta os valores para todas as letras maiúsculas,
- remova '$' e ',' da coluna Tuition e converta a coluna para o tipo de dados float
- crie a coluna Nome e Sobrenome a partir do Nome.

In [25]:
df2 = pd.DataFrame({'Name':['Kim Mend', 'Rob Brun', 'Lin Lu', 'Ron Guy'],
                  'Gender':['Femal  ', '  Male  ', 'male', 'Male'],
                   'Birthday':['3/31/1998', 'Jan 15, 1999', '20/1/1998', '5/6/1997'],
                  'Tuition':['$10,125', '$9999', '0', '$2000']})
df2

,Name,Gender,Birthday,Tuition
0,Kim Mend,Femal,3/31/1998,"$10,125"
1,Rob Brun,Male,"Jan 15, 1999",$9999
2,Lin Lu,male,20/1/1998,0
3,Ron Guy,Male,5/6/1997,$2000


Para remover espaços em branco à esquerda/à direita e uniformizar maiúsculas e minúsculas, utilizamos os métodos `strip()` e `upper()`, respectivamente.

In [26]:
#Strip leading/trailing spaces in Gender column
df2['Gender'] = df2.Gender.str.strip()
#Convert to all upper case
df2['Gender'] = df2.Gender.str.upper()
df2

,Name,Gender,Birthday,Tuition
0,Kim Mend,FEMAL,3/31/1998,"$10,125"
1,Rob Brun,MALE,"Jan 15, 1999",$9999
2,Lin Lu,MALE,20/1/1998,0
3,Ron Guy,MALE,5/6/1997,$2000


Os valores da moeda algumas vezes são armazenados como strings como $ 12.345,99. Precisamos retirar o cifrão e a vírgula dos valores e depois converter para float para outras operações. O código a seguir realiza 3 tarefas em uma linha. Observe que você precisará envolver o atributo str para a chamada da função de string `replace()`.

In [27]:
#Remove $ and , in Tuition, convert to float
df2['Tuition'] = df2.Tuition.str.replace('$', '').str.replace(',','').astype(float)
df2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  


,Name,Gender,Birthday,Tuition
0,Kim Mend,FEMAL,3/31/1998,10125.0
1,Rob Brun,MALE,"Jan 15, 1999",9999.0
2,Lin Lu,MALE,20/1/1998,0.0
3,Ron Guy,MALE,5/6/1997,2000.0


## Manipulação de Datetime

Data e hora são muito comuns em conjuntos de dados relacionados a negócios. Sempre queremos converter as informações de data e hora de string para tipo de dados de data e hora. Às vezes, precisamos criar colunas relacionadas a data e hora para análise relacionada a data e hora, ou seja. vendas em diferentes dias da semana.

Converter string de data e hora em objeto de data e hora
Pandas tem uma função `to_datetime()` irá converter a string de data e hora para o tipo de dados datetime. Esta função pode lidar com vários formatos de data e hora. No código a seguir, criaremos uma nova coluna Birthday_clean de Birthday.

In [28]:
df2['Birthday_clean'] = pd.to_datetime(df2.Birthday)
df2

,Name,Gender,Birthday,Tuition,Birthday_clean
0,Kim Mend,FEMAL,3/31/1998,10125.0,1998-03-31
1,Rob Brun,MALE,"Jan 15, 1999",9999.0,1999-01-15
2,Lin Lu,MALE,20/1/1998,0.0,1998-01-20
3,Ron Guy,MALE,5/6/1997,2000.0,1997-05-06


Precisamos ter muito cuidado ao lidar com valores de data e hora, por exemplo, para '5/6/1998', não está claro se é 6 de maio ou 5 de junho. pd.to_datetime() por padrão assumirá que '5' é mês e ' 6' é a data. Mas este pode não ser o caso. Você pode passar o formato de data e hora para a função para eliminar a ambiguidade, como mostra o exemplo a seguir. Mas se você deseja aplicar um formato a uma coluna DataFrame, essa coluna deve ter o mesmo formato de data e hora. Para conjuntos de dados que têm um formato de data e hora muito confuso, você precisa de técnicas mais avançadas para limpá-los, o que está fora do escopo desta lição.

In [29]:
#set datetime format
pd.to_datetime('5/6/1997', format='%d/%m/%Y')

Timestamp('1997-06-05 00:00:00')

#### Criar novas colunas da coluna de data e hora

O objeto datetime do Pandas tem funções para obter a parte de data e hora como ano, mês, dia, dia da semana, hora etc. Semelhante às funções de string, precisamos usar o atributo de data e hora `dt` quando invocar funções de data e hora. Nas células a seguir, adicionaremos novas colunas relacionadas a data e hora no DataFrame criado acima.

In [30]:
df2['Year'] = df2.Birthday_clean.dt.year
df2['Month'] = df2.Birthday_clean.dt.month
df2['Day'] = df2.Birthday_clean.dt.day
df2['DayOfWeek'] = df2.Birthday_clean.dt.dayofweek
df2['Hour'] = df2.Birthday_clean.dt.hour
df2

,Name,Gender,Birthday,Tuition,Birthday_clean,Year,Month,Day,DayOfWeek,Hour
0,Kim Mend,FEMAL,3/31/1998,10125.0,1998-03-31,1998,3,31,1,0
1,Rob Brun,MALE,"Jan 15, 1999",9999.0,1999-01-15,1999,1,15,4,0
2,Lin Lu,MALE,20/1/1998,0.0,1998-01-20,1998,1,20,1,0
3,Ron Guy,MALE,5/6/1997,2000.0,1997-05-06,1997,5,6,1,0


## Função Lambda

As funções lambda do Python, também conhecidas como funções anônimas, são funções embutidas que não têm nome. Eles são criados com a palavra-chave lambda. A sintaxe é `argumentos lambda: expressão`. A célula a seguir define uma função lambda que soma 2 números.

In [31]:
x = lambda a, b: a + b
x(3,4)

7

A função Lambda é muito útil em manipulações de DataFrame. Usaremos 2 exemplos para demonstrar isso. O primeiro é dividido de uma coluna para duas colunas com função lambda. A segunda é criar uma coluna de data e hora de várias colunas.

#### Aplicar função Lambda em uma coluna

A sintaxe para aplicar a função lambda em uma coluna é `df.column.apply(lambda x:expression(x))`. A função `apply` aplicará a função lambda em cada valor na coluna e, em seguida, retornará uma série pandas com o valor de retorno da função lambda. Criaremos a coluna FirstName e Lastname aplicando a função lambda na coluna Name no exemplo a seguir.

In [32]:
#create last name and first name from Name
df2['Firstname'] = df2.Name.apply(lambda x:x.split()[0])
df2['Lastname'] = df2.Name.apply(lambda x:x.split()[1])
df2

,Name,Gender,Birthday,Tuition,Birthday_clean,Year,Month,Day,DayOfWeek,Hour,Firstname,Lastname
0,Kim Mend,FEMAL,3/31/1998,10125.0,1998-03-31,1998,3,31,1,0,Kim,Mend
1,Rob Brun,MALE,"Jan 15, 1999",9999.0,1999-01-15,1999,1,15,4,0,Rob,Brun
2,Lin Lu,MALE,20/1/1998,0.0,1998-01-20,1998,1,20,1,0,Lin,Lu
3,Ron Guy,MALE,5/6/1997,2000.0,1997-05-06,1997,5,6,1,0,Ron,Guy


No exemplo acima, a função lambda `lambda x:x.split()[0]` recebe um argumento de string, x, `x.split()[0]` primeiro chamará a função split em x que divide x com espaço em branco, então pegue a primeira palavra, que é o primeiro nome. Da mesma forma, a segunda palavra é sobrenome.

#### Aplicar função Lambda em linhas/colunas

Você também pode aplicar a função lambda nas linhas ou colunas do DataFrame com a sintaxe `df.apply(lambda x: expression(x), axis=1)`. Observe que há um novo argumento `axis`. Isso ocorre porque ao chamar apply() em um DataFrame, x será uma coluna ou uma linha do DataFrame. `axis=0` significa que x será uma coluna do DataFrame. Na maioria dos casos, aplicamos a função lambda em cada linha, o que significa `axis=1`. No exemplo a seguir, criaremos uma nova coluna de data e hora com base na coluna de ano, mês e dia.

In [33]:
from datetime import datetime
df2['Birthday_created'] = df2.apply(lambda x: datetime(x.Year, x.Month, x.Day), axis=1)
df2

,Name,Gender,Birthday,Tuition,Birthday_clean,Year,Month,Day,DayOfWeek,Hour,Firstname,Lastname,Birthday_created
0,Kim Mend,FEMAL,3/31/1998,10125.0,1998-03-31,1998,3,31,1,0,Kim,Mend,1998-03-31
1,Rob Brun,MALE,"Jan 15, 1999",9999.0,1999-01-15,1999,1,15,4,0,Rob,Brun,1999-01-15
2,Lin Lu,MALE,20/1/1998,0.0,1998-01-20,1998,1,20,1,0,Lin,Lu,1998-01-20
3,Ron Guy,MALE,5/6/1997,2000.0,1997-05-06,1997,5,6,1,0,Ron,Guy,1997-05-06
